# 9.1 Optimising with pure Python

### TODO:

- [ ] rename mandel to mandel_baseline?
- [ ] instructions for installing line_profiler
- [ ] More on dis

*Estimated time for this notebook: 5 minutes*

We start by copying our `mandel()` function into this notebook and reproducing our list of input values for the function.

In [1]:
def mandel(constant, max_iterations=50):
    """Computes the values of the series for up to a maximum number of iterations. 
    
    The function stops when the absolute value of the series surpasses 2 or when it reaches the maximum 
    number of iterations.
    
    Returns the number of iterations.
    """
    
    value = 0

    counter = 0
    while counter < max_iterations:
        if abs(value) > 2:
            break

        value = (value ** 2) + constant

        counter = counter + 1

    return counter


assert mandel(0) == 50
assert mandel(3) == 1
assert mandel(0.5) == 5

In [2]:
xmin = -1.5
ymin = -1.0
xmax = 0.5
ymax = 1.0
resolution = 300
xstep = (xmax - xmin) / resolution
ystep = (ymax - ymin) / resolution
xs = [(xmin + xstep * i) for i in range(resolution)]
ys = [(ymin + ystep * i) for i in range(resolution)]

## Attempt 1: For-loop vs. list comprehension

We want to run our `mandel` function on a range of complex input values. We can do that by either looping over all the values in the (nested) list, or by using a list comprehension as we did in the previous notebook. We know that list comprehensions are supposed to be more "pythonic", but are they also faster? Let's find out!

In [3]:
def listcomp_mandel(xs, ys):
    return [[mandel(complex(x, y)) for x in xs] for y in ys]

In [4]:
def for_loop_mandel(xs, ys):
    result = []
    
    for y in ys:
        for x in xs:
            result.append(mandel(complex(x, y)))
    return result

In [5]:
%%timeit
listcomp_mandel(xs, ys)

430 ms ± 10.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit
for_loop_mandel(xs, ys)

424 ms ± 10.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The for loops are slightly slower, but the difference is really small. A bit disappointing, but at least the list comprehensions look more elegant.

## Profiling

It looks like we don't gain much by changing the way we call the function. Maybe we should instead focus on the function itself. 
There are useful tools out there that we can use to gain more insight into which part of a function takes up most computation time. In this module we use `line_profiler`.

`line_profiler` is a Python module that we need to install into our virtual environment before we can use it.
TODO: installation instructions.
This tool breaks down the computation time of our function line by line. 

In [7]:
%load_ext line_profiler

In [8]:
%lprun -f mandel mandel(complex(xs[0], ys[0]))

## Attempt 2: Multiplication vs. power

Looking at the `line_profiler` output, it seems like the line where we compute the value of the series for the current iteration takes quite long. The operations that are performed are squaring the value (using power 2) and adding a constant. Maybe we can speed up the code by replacing the power operation by multiplying the value with itself. 

Before we re-write the `mandel()` function, we will write new functions that repeatedly square a value for a given number of iterations.

In [9]:
def square_power(value, iterations):
    """Repeatedly square a value using **."""
    
    for _ in range(iterations):
        value = value ** 2
        
    return value
        
assert square_power(2, 1) == 4
assert square_power(2, 0) == 2

## Exercise 9.2

- Write another function that does the same, but squares a value by multiplying it with itself.
- Compare the performance of the two functions. Which one is faster?

## Solution: Exercise 9.2

In [10]:
def square_multiply(value, iterations):
    """Repeatedly square a value using *."""
    
    for _ in range(iterations):
        value = value * value
    
    return value

assert square_multiply(2, 1) == 4
assert square_multiply(2, 0) == 2

In [11]:
%%timeit
square_power(1.9, 10)

868 ns ± 30.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [12]:
%%timeit
square_multiply(1.9, 10)

533 ns ± 7.53 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


The computational cost for `square_power` and `square_multiply` are quite different. 
Conclusion: `**` is not simply a wrapper around `*`.

Can we find out more about this?

In [13]:
import dis

In [14]:
dis.dis(square_power)

  4           0 SETUP_LOOP              24 (to 26)
              2 LOAD_GLOBAL              0 (range)
              4 LOAD_FAST                1 (iterations)
              6 CALL_FUNCTION            1
              8 GET_ITER
        >>   10 FOR_ITER                12 (to 24)
             12 STORE_FAST               2 (_)

  5          14 LOAD_FAST                0 (value)
             16 LOAD_CONST               1 (2)
             18 BINARY_POWER
             20 STORE_FAST               0 (value)
             22 JUMP_ABSOLUTE           10
        >>   24 POP_BLOCK

  7     >>   26 LOAD_FAST                0 (value)
             28 RETURN_VALUE


In [15]:
dis.dis(square_multiply)

  4           0 SETUP_LOOP              24 (to 26)
              2 LOAD_GLOBAL              0 (range)
              4 LOAD_FAST                1 (iterations)
              6 CALL_FUNCTION            1
              8 GET_ITER
        >>   10 FOR_ITER                12 (to 24)
             12 STORE_FAST               2 (_)

  5          14 LOAD_FAST                0 (value)
             16 LOAD_FAST                0 (value)
             18 BINARY_MULTIPLY
             20 STORE_FAST               0 (value)
             22 JUMP_ABSOLUTE           10
        >>   24 POP_BLOCK

  7     >>   26 LOAD_FAST                0 (value)
             28 RETURN_VALUE


### Attempt 3: In-place multiplication

Python has an operator for inplace multiplication: `*=`. Is that faster?

In [16]:
def square_inplace_multiply(value, iterations):
    """Repeatedly square a value using *."""
    
    for _ in range(iterations):
        value *= value 
    
    return value

assert square_multiply(2, 1) == 4
assert square_multiply(2, 0) == 2

In [17]:
%%timeit
square_inplace_multiply(1.9, 10)

552 ns ± 11.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


This is the slowest way of squaring a value that we've tried.

In [18]:
dis.dis(square_inplace_multiply)

  4           0 SETUP_LOOP              24 (to 26)
              2 LOAD_GLOBAL              0 (range)
              4 LOAD_FAST                1 (iterations)
              6 CALL_FUNCTION            1
              8 GET_ITER
        >>   10 FOR_ITER                12 (to 24)
             12 STORE_FAST               2 (_)

  5          14 LOAD_FAST                0 (value)
             16 LOAD_FAST                0 (value)
             18 INPLACE_MULTIPLY
             20 STORE_FAST               0 (value)
             22 JUMP_ABSOLUTE           10
        >>   24 POP_BLOCK

  7     >>   26 LOAD_FAST                0 (value)
             28 RETURN_VALUE


### Mandelbrot "square-by-multiply"

Let's rewrite the `mandel()` function and time its performance, using what we've learnt in this notebook:
- Square the value by multiplication with itself (but not inplace).
- Run the function over a range of values by using list comprehension (instead of for-loop).

In [19]:
def mandel_square_multiply(constant, max_iterations=50):
    """Computes the values of the series for up to a maximum number of iterations. 
    
    The function stops when the absolute value of the series surpasses 2 or when it reaches the maximum 
    number of iterations.
    
    Returns the number of iterations.
    """
    
    value = 0

    counter = 0
    while counter < max_iterations:
        if abs(value) > 2:
            break

        value = (value * value) + constant

        counter = counter + 1

    return counter


assert mandel(0) == 50
assert mandel(3) == 1
assert mandel(0.5) == 5

In [21]:
%%timeit
[[mandel_square_multiply(complex(x, y)) for x in xs] for y in ys]

373 ms ± 4.72 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Things to be aware of

- Try to understand _why_ one method is faster than the other.
- Is it worth the speed up? 
- Do you get the same answers for all possible inputs?
- We're using CPython. Would the optimisations still hold for other implementations of Python?
